In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import re
from camel_tools.utils.charmap import CharMapper
sys.path.insert(0,"..")
from src.predict import translit_rules
from src.evaluation.evaluate import evaluate_df
from src.data import make_dataset
from camel_tools.utils.normalize import normalize_unicode


In [2]:
pd.set_option("display.max_rows", 100)

In [3]:
ar2safebw = CharMapper.builtin_mapper('ar2safebw')

In [4]:
# validating mappings
pd.set_option('display.max_rows', 5)

caphimap = translit_rules.load_caphi_mappings()
locmap = translit_rules.load_loc_mappings()
locexceptional = translit_rules.load_exceptional_spellings()
mapping = pd.read_csv(Path(os.getcwd()).parent.joinpath('src','predict','ar2phon','ar2phon_map.tsv'),delimiter='\t').fillna('')

# # copy safebw to map file for easy readability of rules
# mapping['Arabic'].apply(ar2safebw).to_clipboard(index=False,header=False)

validation = pd.read_csv(Path(os.getcwd()).parent.joinpath('src','predict','ar2phon','validate_mappings.tsv'),delimiter='\t').fillna('')
validation['predict_caphi'] = validation['arabic'].apply(lambda x: translit_rules.translit_caphi_token(x,caphimap))
validation['predict_loc'] = validation['arabic'].apply(lambda x: translit_rules.translit_loc_token(x,locmap,locexceptional))
validation['match_caphi'] = validation.apply(lambda x: x['caphi']==x['predict_caphi'],axis=1)
validation['match_loc'] = validation.apply(lambda x: x['loc']==x['predict_loc'],axis=1)
validation.value_counts(['match_loc','match_caphi']) # 3 will always return false by design

match_loc  match_caphi
True       True           94
False      False           3
dtype: int64

In [5]:
translit_rules.translit_loc_sentence('دراسة للفکر السيسى لطلاب المدرسة الثانوية العامة/',locmap,locexceptional)

'Drāsh llfکr alsysá lṭlāb almdrsh althānwyh alʻāmh/'

In [3]:
dev = pd.read_csv(f'{Path(os.getcwd()).parent}/data/processed/dev.tsv',delimiter='\t')


In [103]:
predictions = pd.read_table('/Users/fae211/ba3sasah/LOC_transcribe/predictions_out/translit_rules/dev.tsv',header=None)[0]


0                       Rāfʻy, ʻbd alrḥmn.
1        Mqdmāt thwrh 23 ywlywā snh 1952 :
                       ...                
61717                            Dār ṭlās,
61718             Ṣfḥāt mn tārykh albʻth ;
Name: 0, Length: 61719, dtype: object

In [104]:
head = dev[:100]
head['predict_translit_simple'] = predictions[:100]

In [89]:
head['predict_translit_simple'] = head['ar'].apply(lambda x: translit_rules.translit_loc_sentence(x,locmap,locexceptional))
# head[['rom','predict-rom']]

In [105]:
evaluate_head = evaluate_df(head)#,eval_funcs=[match_ignore_case])

In [4]:
# # capitalization errors by tag
# for tag in evaluate_head['comb.tag'].drop_duplicates():
#     print(tag)
#     print(evaluate_head[(evaluate_head['comb.tag']==tag)&(evaluate_head.apply(lambda x: x['match_ignore_case']==True,axis=1))].value_counts(['match'],normalize=True).to_string())

In [57]:
def contains(sent,charset):
    for char in charset:
        if char in sent:
            return True
    return False

In [6]:
evaluation = pd.read_csv(Path(os.getcwd()).parent.joinpath('evaluation/translit_simple.tsv'),delimiter='\t')

In [15]:
evaluation[:100]

,gold_tok,source_tok,predict_translit_simple_tok,sentID,recID,comb.tag,match,match_ignore_case
0,"Rāfiʻī,",رافعي،,"Rāfʻy,",0,umich-1,100a,False,False
1,ʻAbd,عبد,ʻbd,0,umich-1,100a,False,False
2,al-Raḥmān.,الرحمن.,alrḥmn.,0,umich-1,100a,False,False
3,Muqaddimāt,مقدمات,Mqdmāt,1,umich-1,245a,False,False
4,thawrat,ثورة,thwrh,1,umich-1,245a,False,False
5,23,23,23,1,umich-1,245a,True,True
6,Yūlyū,يوليوا,ywlywā,1,umich-1,245a,False,False
7,sanat,سنة,snh,1,umich-1,245a,False,False
8,1952,1952,1952,1,umich-1,245a,True,True
9,:,:,:,1,umich-1,245a,True,True
